# Notebook Imports

In [1]:
import pandas as pd
import numpy as np

# Constants

In [73]:
VOCAB_SIZE = 2500

TRAINING_DATA_FILE = "SpamData/SpamData/02_Training/train-data.txt"
TEST_DATA_FILE = "SpamData/SpamData/02_Training/test-data.txt"

TOKEN_SPAM_PROB_FILE = "SpamData/SpamData/03_Testing/prob-spam.txt"
TOKEN_HAM_PROB_FILE = "SpamData/SpamData/03_Testing/prob-nonspam.txt"
TOKEN_ALL_PROB_FILE = "SpamData/SpamData/03_Testing/prob-all-tokens.txt"

TEST_FEATURE_MATRIX = "SpamData/SpamData/03_Testing/test-features.txt"
TEST_TARGET_FILE = "SpamData/SpamData/03_Testing/test-target.txt"

# Read and Load Features from .txt Files into NumPy Array

In [3]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype=int)

In [4]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)

In [5]:
sparse_train_data[:5]

array([[ 0,  2,  1,  1],
       [ 0,  3,  1,  2],
       [ 0,  4,  1,  1],
       [ 0,  7,  1,  3],
       [ 0, 11,  1,  1]])

In [6]:
sparse_train_data[-5:]

array([[5795, 2032,    0,    2],
       [5795, 2037,    0,    1],
       [5795, 2271,    0,    1],
       [5795, 2348,    0,    1],
       [5795, 2379,    0,    1]])

In [7]:
print("Number of rows in training file", sparse_train_data.shape[0])
print("Number of rows in test file", sparse_test_data.shape[0])

Number of rows in training file 258332
Number of rows in test file 117774


In [8]:
print("Number of emails in training file", np.unique(sparse_train_data[:, 0]).size)

Number of emails in training file 4013


In [9]:
print("Number of emails in test file", np.unique(sparse_test_data[:, 0]).size)

Number of emails in test file 1724


## How to create an Empty DataFrame

In [10]:
column_names = ["DOC_ID"] + ["CATEGORY"] + list(range(0, VOCAB_SIZE))

In [11]:
column_names[:18]

['DOC_ID', 'CATEGORY', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [12]:
len(column_names)

2502

In [13]:
index_names = np.unique(sparse_train_data[:, 0])

In [14]:
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [15]:
full_train_data = pd.DataFrame(index=index_names, columns=column_names)
full_train_data.fillna(value=0, inplace=True)

In [16]:
full_train_data.tail()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
5789,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5794,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5795,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
sparse_train_data[10:13]

array([[ 0, 31,  1,  1],
       [ 0, 33,  1,  1],
       [ 0, 45,  1,  1]])

In [18]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    Form a full matrix a sparse matrix. Return a pandas dataframe.
    
    Keyword arguments:
    sparse_matrix -- numpy array
    nr_words -- size of the vocabulary. Total number of tokens.
    doc_idx -- position of the document id in the sparse matrix. Default: 1st column
    word_idx -- position of the word id in the sparse matrix. Default: 2 nd column
    cat_idx -- position od the label (spam is 1, nonspam is 0). Default: 3rd column
    freq_idx -- position of occurrence of word in sparse matrix. Default: 4th column
    """
    column_names = ["DOC_ID"] + ["CATEGORY"] + list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    full_matrix= pd.DataFrame(index=doc_id_names, columns=column_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, "DOC_ID"] = doc_nr
        full_matrix.at[doc_nr, "CATEGORY"] = label
        full_matrix.at[doc_nr, word_id] = occurrence

    full_matrix.set_index("DOC_ID", inplace=True)
    return full_matrix

In [19]:
%%time
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

CPU times: user 6.03 s, sys: 105 ms, total: 6.14 s
Wall time: 6.13 s


In [20]:
full_train_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,1,2,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6,0,0,2,4,0,3,14,0,...,0,0,0,0,0,0,0,0,0,0
4,1,5,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


# Training the Naive Bayes Model
## Calculating the Probability of Spam

In [21]:
full_train_data.CATEGORY.size

4013

In [22]:
full_train_data.CATEGORY.sum()

1248

In [23]:
#Calculating the probability of spam
prob_spam = full_train_data.CATEGORY.sum()/full_train_data.CATEGORY.size
print("Probability of spam is: %.0f"%(prob_spam*100) + "%")

Probability of spam is: 31%


## Tokens Number of Words / Tokens

In [24]:
full_train_features = full_train_data.loc[:, full_train_data.columns != "CATEGORY"]

In [25]:
full_train_features

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,3,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,1,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,3,1,0,1,1,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0


In [26]:
email_lengths = full_train_features.sum(axis=1)

In [27]:
email_lengths

DOC_ID
0        87
1        53
2        40
3       183
4        43
       ... 
5789     18
5790     72
5791     46
5794     57
5795    264
Length: 4013, dtype: int64

In [28]:
total_wc = email_lengths.sum()

In [29]:
total_wc

429184

## Number of Tokens in Spam & Ham emails

In [30]:
ham_lengths = email_lengths[full_train_data.CATEGORY == 0]
spam_lengths = email_lengths[full_train_data.CATEGORY == 1]

In [31]:
ham_lengths.head()

DOC_ID
1896     82
1898    112
1899     66
1900     49
1901     34
dtype: int64

In [32]:
ham_lengths.shape

(2765,)

In [33]:
ham_wc = ham_lengths.sum()

In [34]:
ham_wc

252866

In [35]:
spam_lengths.head()

DOC_ID
0     87
1     53
2     40
3    183
4     43
dtype: int64

In [36]:
spam_lengths.shape

(1248,)

In [37]:
spam_wc = spam_lengths.sum()

In [38]:
spam_wc

176318

In [39]:
#cheking if subsets are correct
email_lengths.shape[0] - spam_lengths.shape[0] - ham_lengths.shape[0]

0

In [40]:
nonspam_wc = ham_lengths.sum()

In [41]:
nonspam_wc

252866

In [42]:
spam_wc + nonspam_wc - total_wc

0

In [43]:
print(f"Average number of words in spam emails {spam_wc / spam_lengths.shape[0]}")

Average number of words in spam emails 141.28044871794873


In [44]:
print(f"Average number of words in ham emails {ham_wc / ham_lengths.shape[0]}")

Average number of words in ham emails 91.45244122965642


## Summing the Tokens Occuring in Spam

In [45]:
full_train_features.shape

(4013, 2500)

In [46]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]

In [47]:
train_spam_tokens.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
train_spam_tokens.shape

(1248, 2500)

In [49]:
# Laplace smoothing technic
summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1

In [50]:
summed_spam_tokens

0       2179
1        935
2       1217
3       2022
4       1219
        ... 
2495       8
2496       1
2497       2
2498       1
2499      26
Length: 2500, dtype: int64

## Summing the Tokens Ocuuring in Ham

In [51]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]

In [52]:
train_ham_tokens

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1896,1,2,1,0,1,1,2,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1898,2,1,0,1,1,0,2,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1899,2,1,1,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1900,3,4,0,1,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1901,2,2,0,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,3,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,1,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,3,1,0,1,1,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0


In [53]:
train_ham_tokens.shape

(2765, 2500)

In [54]:
# Laplace smoothing technic
summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1

In [55]:
summed_ham_tokens

0       5484
1       2590
2       2045
3        938
4       1612
        ... 
2495      27
2496       6
2497      28
2498      25
2499       3
Length: 2500, dtype: int64

In [56]:
train_ham_tokens[2499].sum() + 1

3

## P(Token | Spam) - Probability that a Token Occurs given the Emails is Spam

In [57]:
print(f"{spam_wc} " + f" {VOCAB_SIZE}")

176318  2500


In [58]:
prob_tokens_spam = summed_spam_tokens / (spam_wc + VOCAB_SIZE)

In [59]:
prob_tokens_spam

0       0.012186
1       0.005229
2       0.006806
3       0.011308
4       0.006817
          ...   
2495    0.000045
2496    0.000006
2497    0.000011
2498    0.000006
2499    0.000145
Length: 2500, dtype: float64

In [60]:
prob_tokens_spam.sum()

1.0

## P(Token | Ham) - Probability that a Token Occurs given the Emails is Nonspam

In [61]:
print(f"{ham_wc} " + f" {VOCAB_SIZE}")

252866  2500


In [62]:
prob_tokens_ham = summed_ham_tokens / (nonspam_wc + VOCAB_SIZE)

In [63]:
# ham_wc == nonspam_wc

In [64]:
prob_tokens_ham

0       0.021475
1       0.010142
2       0.008008
3       0.003673
4       0.006313
          ...   
2495    0.000106
2496    0.000023
2497    0.000110
2498    0.000098
2499    0.000012
Length: 2500, dtype: float64

In [65]:
prob_tokens_ham.sum()

1.0

## P(Token) - Probability that Token Occurs

In [67]:
prob_tokens_all = full_train_features.sum(axis=0) / total_wc

In [68]:
prob_tokens_all

0       0.017850
1       0.008209
2       0.007596
3       0.006892
4       0.006592
          ...   
2495    0.000077
2496    0.000012
2497    0.000065
2498    0.000056
2499    0.000063
Length: 2500, dtype: float64

In [69]:
prob_tokens_all.sum()

1.0

## Save the Trained Model

In [72]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_ham)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

## Prepare Test Data

In [76]:
sparse_test_data.shape

(117774, 4)

In [77]:
%%time

full_test_data = make_full_matrix(sparse_test_data, nr_words=VOCAB_SIZE)

CPU times: user 3.95 s, sys: 24.4 ms, total: 3.98 s
Wall time: 3.98 s


In [78]:
full_test_data

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
8,1,0,0,1,4,2,1,2,4,1,...,0,0,0,0,0,0,0,0,0,0
12,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
14,1,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,1,0,2,1,1,2,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
17,1,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5783,0,2,1,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5786,0,5,5,2,2,1,2,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5788,0,0,4,0,2,4,3,3,1,4,...,0,0,0,0,0,0,0,0,0,0


In [82]:
X_test = full_test_data.loc[:, full_test_data.columns != "CATEGORY"]
y_test = full_test_data.CATEGORY

In [83]:
np.savetxt(TEST_TARGET_FILE, y_test)
np.savetxt(TEST_FEATURE_MATRIX, X_test)